Importation des librairies utiles

In [38]:
import requests
import pandas as pd

Création du dictionnaire avec les différentes catégories de signalements et leur numéro

In [39]:
categories_list = [
    {"catcolor": "paleturquoise", "catid": 8, "catname": "Absence d'aménagement", "catname_en_US": "Missing of bicycle or pedestrian way"},
    {"catcolor": "seagreen", "catid": 3, "catname": "Aménagement mal conçu", "catname_en_US": "Bad designed way"},
    {"catcolor": "darkgreen", "catid": 4, "catname": "Défaut d'entretien", "catname_en_US": "Maintenance issue"},
    {"catcolor": "turquoise", "catid": 5, "catname": "Absence d'arceaux de stationnement", "catname_en_US": "Missing bicycle parking"},
    {"catcolor": "crimson", "catid": 2, "catname": "Véhicule ou objet gênant", "catname_en_US": "Annoying vehicle or object"},
    {"catcolor": "black", "catid": 9, "catname": "Accident, chute, incident", "catname_en_US": "Accident, incident or drop "},
    {"catcolor": "yellowgreen", "catid": 6, "catname": "Signalisation, marquage", "catname_en_US": "Traffic sign or labeling"},
    {"catcolor": "lightcoral", "catid": 7, "catname": "Incivilité récurrente sur la route", "catname_en_US": "Antisocial behaviour on the road"},
    {"catcolor": "blue", "catid": 10, "catname": "Vol ou dégradation de vélo", "catname_en_US": "Bike theft or vandalism"},
    {"catcolor": "grey", "catid": 100, "catname": "Autre", "catname_en_US": "Other"},
    {"catcolor": "orange", "catdisable": True, "catid": 50, "catname": "COVID - Distanciation physique impossible"},
    {"catcolor": "red", "catdisable": True, "catid": 51, "catname": "COVID - Aménagement fermé"},
    {"catcolor": "green", "catdisable": True, "catid": 52, "catname": "COVID - Voirie en période de confinement"},
    {"catcolor": "black", "catid": 11, "catname": "Éclairage public insuffisant", "catname_en_US": "Insufficient public lighting"}
]

# Convertir la liste de catégories en un dictionnaire avec le nom de la catégorie comme clé et l'identifiant de catégorie comme valeur
categories_dict = {cat["catname"]: str(cat["catid"]) for cat in categories_list}

# Création du dataframe des catégories avec les colonnes "id" et "catégorie"
df_categories = pd.DataFrame(categories_list).loc[:, ["catid", "catname"]]

# Renommer les colonnes
df_categories = df_categories.rename(columns={"catid": "id", "catname": "catégorie"})

# Changer l'index et trier par id
df_categories = df_categories.set_index("id").sort_index()

# Afficher le dataframe
df_categories

,catégorie
id,
2,Véhicule ou objet gênant
3,Aménagement mal conçu
4,Défaut d'entretien
5,Absence d'arceaux de stationnement
6,"Signalisation, marquage"
7,Incivilité récurrente sur la route
8,Absence d'aménagement
9,"Accident, chute, incident"
10,Vol ou dégradation de vélo


Initialisation des paramètres

In [40]:
# Paramètres de recherche

# Filtre de recherche par date de début, exemple : start_date = "2021-01-01"
start_date = None #Default value

# Filtre de recherche par date de fin, exemple : end_date = "2021-12-31"
end_date = None #Default value

# url de l'API
url = 'https://vigilo.bapav.org/get_issues.php'

# Filtre de recherche par localisation, exemple : location_filter = "Brest"
location_filter = 'Bordeaux'  # Default value

# Filtre de recherche par statut, exemple : status_filter = "En cours"
status_filter = None  # Default value

# Filtre de recherche par approbation, exemple : approval_filter = "Approuvé"
approval_filter = None  # Default value

# Filtre de recherche par périmètre, exemple : scope_filter = "Bordeaux"
scope_filter = None # Default value

# Filtre de recherche par mot-clé, exemple : keyword_search = "piste cyclable"
keyword_search = None  # Default value

# Limite de données à afficher, exemple : data_limit = 100
data_limit = None  # Default value

# Filtre des catégories à afficher, exemple : category_filter = categories_dict["Absence d'aménagement"]
# donne la catégorie "Absence d'aménagement" 
category_filter = None

Fonction créant le dataframe

In [41]:
def data_collection(start_date, end_date, url, category_filter=None, location_filter=None,
                    status_filter=None, approval_filter=None, scope_filter=None,
                    keyword_search=None, data_limit=None):
    try:
        params = {
            'start_date': start_date,
            'end_date': end_date,
            'location_filter': location_filter,
            'status_filter': status_filter,
            'approval_filter': approval_filter,
            'scope_filter': scope_filter,
            'keyword_search': keyword_search,
            'data_limit': data_limit
        }
        resp = requests.get(url, params=params)
        resp.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
        df = pd.DataFrame(resp.json())
        return df[df['categorie'] == category_filter] if category_filter else df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
        return None

Création du dataframe

In [42]:
df = data_collection(start_date, end_date, url, category_filter, location_filter,
                     status_filter, approval_filter, scope_filter, keyword_search, data_limit)

test

In [43]:
df

,token,coordinates_lat,coordinates_lon,address,comment,explanation,time,status,group,categorie,approved,cityname
0,DD7230F6,48.41161342100428,-4.489712102838329,"Rue Honoré de Balzac, Brest",Stationnement sur trottoir VS pietons,,1703227560,0,0,2,1,Brest
1,7FA00E08,48.4123682974901,-4.4859784307144155,"Rue Étienne Hubac, Brest",Stationnement sur trottoir VS pietons,Une rue plus qu\'assez large pour stationner d...,1703227320,0,0,2,1,Brest
2,AFCA2E97,48.39148377417681,-4.469872824847698,"Pierre Semard , Brest",stationnement sur la piste cyclable,,1703184300,0,0,2,1,Brest
3,B7F5BC01,48.39149290224677,-4.469862096011639,"Pierre Semard , Brest",Stationnement sur piste cyclabe,,1703097780,0,0,2,1,Brest
4,3D5F904D,48.410124999629836,-4.483387349426027,"Rue Kertatupage, Brest",Stationnement sur trottoir VS pietons,Aucun passage n\'est plus possible pour un piéton,1703070600,0,0,2,1,Brest
...,...,...,...,...,...,...,...,...,...,...,...,...
796,D6E5135E,48.38702,-4.49049,"Rue de Siam, Brest",Intersection rue de Siam rue d'Aiguillon.,,1563797280,3,0,5,1,Brest
797,39548EF5,48.38859543703247,-4.515320844948292,"Rue Dupuy de Lôme, Brest",il ne reste que la place pour une poussette ca...,,1563377220,0,0,2,1,Brest
798,A8619257,48.38683314356656,-4.528028815984726,"Boulevard de Plymouth, Brest",croisement pc et chemin piéton non explicite,,1561998120,0,0,3,1,Brest
799,593E65B3,48.40313239691792,-4.45753963293432,"Rue du Bot, Brest",Cette bande cyclable sert régulièrement de parki,Bibus ET Breizhgo sont concernés,1561040700,0,0,2,1,Brest
